In [1]:
"""A module that makes downloading and using many GSE datasets fast and easy.

Author: Joshua Blanchard, last update: 4/28/2021

A typical workflow:

    Download wanted GSE family files using the download() function.

    Extract content from these files using the extract() function.

    Use the family_dict() function to convert data to pandas.DataFrame objects.

    With the info() function, perform data analysis on the previously created pandas.DataFrame objects.
"""

'A module that makes downloading and using many GSE datasets fast and easy.\n\nAuthor: Joshua Blanchard, last update: 4/28/2021\n\nA typical workflow:\n\n    Download wanted GSE family files using the download() function.\n\n    Extract content from these files using the extract() function.\n\n    Use the family_dict() function to convert data to pandas.DataFrame objects.\n\n    With the info() function, perform data analysis on the previously created pandas.DataFrame objects.\n'

In [2]:
import pandas as pd
import os
import numpy as np
import re as re
import xmltodict
import shutil
import ftplib
import tarfile
import pickle
import gzip
# import requests
# import wget

In [3]:
__base_path = "data"

In [4]:
__hidden_path = ".aidp_files"

In [5]:
def __family_path(GSE_family):
    
    """
    Exists to build paths to the family's directory.
    
    I downloaded the files using the download() function in conjuction with the extract() function.
    """
        
    return os.path.join(__base_path, GSE_family + "/")


In [6]:
def __clean(file_path):
    
    """
    Cleans a given .txt file.
    
    Returns a dictionary:
    
    "site": first column
    "measurement": second column
    "bad_rows": list of all the invalid rows
    """

    valid_rows = []
    not_valid_rows = []
    file = open(file_path, 'r')
    
    for line in file:
        
#         checks for only the first two columns
        line_match = re.match(r"\S+\t\S+", line)
        if line_match:
            valid_rows.append(line_match.group(0))
        else:
            not_valid_rows.append(line)
        
    file.close()
    
#     now let's split our valid_rows list into two lists, one for each column
    col_1 = []
    col_2 = []
    for row in valid_rows:
        row_match = re.match(r"(\S+)\t(\S+)", row)
        col_1.append(row_match.group(1))
        col_2.append(row_match.group(2))
        
    return {"col_1":col_1, "col_2":col_2, "bad_rows":not_valid_rows}


In [7]:
def __load_file(file_directory):
#     clean data file
#     convert to a dataframe object and return

    """
    Given a file name will output a corresponding pandas.DataFrame object.
    """
    
    try:
        clean_dict = __clean(file_directory)
    except PermissionError:
        print("You likely inputted the path of a directory, not a file.")
    
#     return pd.DataFrame({"site": clean_dict["col_1"], "measurement": clean_dict["col_2"]})
    return pd.DataFrame(data= clean_dict["col_2"], index= clean_dict["col_1"], columns= ["measurement"])


In [8]:
def family_dict(GSE_family):
    
    """
    Given a family ID, will output a dictionary. Keys will be the sample IDs, values will be the corresponding
    pandas.DataFrame object.
    """

#     let's check if we already have this dictionary saved
    if not os.path.exists("./" + __hidden_path):
        os.mkdir(__hidden_path)
    
    dict_path = __hidden_path + "/" + GSE_family + "_dict"    
    if not os.path.exists(dict_path):
    
        family_directory = __family_path(GSE_family)
        total_list = os.listdir(family_directory)
        valid_files = []

        for file_name in total_list:
            match = re.match(r"GSM", file_name)
            if match:
                valid_files.append(file_name)

        family_dict = {}
        for file_name in valid_files:
            file_df = __load_file(os.path.join(family_directory, file_name))
            sample_id = re.match(r"GSM\d+", file_name).group(0)
            family_dict[sample_id] = file_df
            
        dict_file = open(dict_path, 'wb')
        pickle.dump(family_dict, dict_file)
        dict_file.close()
        
    else:
        dict_file = open(dict_path, 'rb')
        family_dict = pickle.load(dict_file)
        dict_file.close()
        
        
    return family_dict


In [9]:
def __matrix_helper(file_path, use= "series()"):
    
    """Returns a tuple containing the start line for reading (0) and the number of rows to read (1).
    
    Possible values for use: "series()", "info()"
    """
    
    file = open(file_path)
    
    line_num = 0
    if use == "series()":
        
        while True:
            line = file.readline()
            
            if "!series_matrix_table_begin" in line:
                start_row = line_num
            elif "!series_matrix_table_end" in line:
                end_row = line_num - 1
            elif line == "":
                break

            line_num += 1
        
    else:
                
        while True:
            line = file.readline()

            if "!Sample_title" in line:
                start_row = line_num
            elif "!series_matrix_table_begin" in line:
                end_row = line_num - 1
            elif line == "":
                break

            line_num += 1

    num_rows = end_row - start_row - 1
        
    return start_row, num_rows

In [10]:
def __matrix_to_df(file_path, use= "series()"):
    
    """Returns the pandas.dataframe corresponding to the series_matrix file.
    
    Possible values for use: "series()", "info()"
    """
    
    start_row, num_rows = __matrix_helper(file_path, use)
    df = pd.read_csv(file_path, header= start_row, sep= "\t", low_memory= False, nrows= num_rows)
    
    if use == "series()":
        df.set_index("ID_REF", inplace= True)
    else:
        df.set_index("!Sample_geo_accession", inplace= True)
        df = df.loc["!Sample_characteristics_ch1"]
    
    return df

In [11]:
def info(GSE_family, sample_id, info= "age"):
    
    """
    Given a GSE family and the ID of the wanted sample will return the desired information of the sample.
    
    Possible values for the info parameter:
    
    "age": the unit of the outputted age will always be years.
    """
#     create dataframe of sample characteristics portion of the series_matrix file
# read desired info from this dataframe. will have to use regex to find the right information
# set up some sort of persistance of this dataframe for faster retrieval of information

#             match = re.search(r"(a|A)(g|G)(e|E)", tag)

    info_df = __matrix_to_df("./" + __base_path + "/" + GSE_family + "/" + GSE_family + "_series_matrix.txt", use= "info()")    
    info_series = info_df.loc[:, sample_id]
        
    for row in info_series:
        match_age = re.search(r"(a|A)(g|G)(e|E)", row)
        if match_age:
    #         let's grab the age
            match_age_dig = re.search(r"\d+\.?\d*", row)
            age_float = float(match_age_dig.group())

    #         now let's find the unit of age
            match_years = re.search(r"(y|Y)(e|E)(a|A)(r|R)(s|S)*", row)
            match_months = re.search(r"(m|M)(o|O)(n|N)(t|T)(h|H)(s|S)*", row)
            match_days = re.search(r"(d|D)(a|A)(y|Y)(s|S)*", row)
            match_hours = re.search(r"(h|H)(o|O)(u|U)(r|R)(s|S)*", row)

    #         we always return the age in units of years
            if match_years:
                return age_float
            elif match_months:
                return age_float / 12
            elif match_days:
                return age_float / 365
            elif match_hours:
                return age_float / 8760
            else:
    #             if no unit is given we'll default to years
                return age_float


In [18]:
def __ID_to_int(GSE_ID):
    
    """
    Given some GSE ID will return the corresponding integer as an int.
    
    Example:
    
    __ID_to_int("GSE41037") will return 41037.
    
    """
    
    match = re.search(r"\d+", GSE_ID)
    
    if not match:
        print(GSE_ID)
    
    return int(match.group(0))


In [19]:
def __sub_directory(GSE_ID):
    
    """
    Given a GSE ID will return the corresponding sub-directory.
    """
    
    gse_int = __ID_to_int(GSE_ID)
    
    if gse_int <= 171:
        ret_str = "GSE" + str(gse_int) + "nnn"
    else:
        first_3_dig = int(str(gse_int)[0:3])
        if first_3_dig <= 171:
            ret_str = "GSE" + str(first_3_dig) + "nnn"
        else:
            first_2_dig_str = str(gse_int)[0:2]
            ret_str = "GSE" + first_2_dig_str + "nnn"
            
    return ret_str


In [20]:
# url = "ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSEnnn/" + "GSE41037" + "/miniml/" + "GSE41027" + "_family.xml.tgz"
# url = "ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSEnnn/GSE1/miniml/GSE1_family.xml.tgz"

def download(GSE_family_list, file_type= "miniml"):
    
    """
    Will download family .tgz files to the following directory: "./data/"
    
    Possible values for file_type: "miniml", "series_matrix"
    """
    
    if (type(GSE_family_list) == str):
        GSE_family_list = [GSE_family_list]

    url = "ftp.ncbi.nlm.nih.gov"
    ftp = ftplib.FTP(url)
    ftp.login()
    
    if not os.path.exists(__base_path):
        os.mkdir(__base_path)
    
    for GSE_family in GSE_family_list:
        
        if file_type == "miniml":
        
            ftp.cwd("/geo/series/" + __sub_directory(GSE_family) + "/" + GSE_family + "/miniml/")
            filename = GSE_family + "_family.xml.tgz"
            
            if not (os.path.exists(__base_path + "/" + filename) or os.path.exists(__base_path + "/" + GSE_family)):
                local_file = open(__base_path + "/" + filename, 'wb')
                ftp.retrbinary('RETR ' + filename, local_file.write, blocksize= 16_384)
                local_file.close()
            
        else:
            
            ftp.cwd("/geo/series/" + __sub_directory(GSE_family) + "/" + GSE_family + "/matrix/")
            filename = GSE_family + "_series_matrix.txt.gz"
        
            if not (os.path.exists(__base_path + "/" + filename) or os.path.exists(__base_path + "/" + GSE_family + "/" + GSE_family + "_series_matrix.txt")):
                local_file = open(__base_path + "/" + filename, 'wb')
                ftp.retrbinary('RETR ' + filename, local_file.write, blocksize= 16_384)
                local_file.close()

    ftp.quit()
    

In [ ]:
def __check_beta(series_df):
    
    """Returns True if Beta values are recorded, returns False if M values are recorded."""

In [21]:
def series(GSE):
    
    """Returns a pandas.DataFrame representative of the series' data."""
    
    download(GSE, file_type= "series_matrix")
    extract()
    
    file_path = os.path.join(__base_path, GSE, GSE + "_series_matrix.txt")
    
    return __matrix_to_df(file_path)
    
    
# download and extract .soft file using the download() and extract() function
# convert file to a dataframe

In [23]:
def __xml_path(GSE_family):
    
    """
    Exists to build a path to the family's .xml.
    
    I downloaded these families using the download() function in conjuction with the extract() function.
    """
    
    return os.path.join(__base_path, GSE_family + "/" + GSE_family + "_family.xml")
    

In [24]:
def __xml_to_dict(xml_path):
    
    """
    Exists to convert a .xml file at xml_path to a dictionary.
    """
    
    family_file = open(xml_path,'r+b')
    family_dict = xmltodict.parse(family_file)
    family_file.close()
    
    return family_dict


In [25]:
def __dict_index(GSE_family, sample_id):
    
    """
    Gives the index of where in the dictionary the sample's information is.
    """
    
    return __sample_indices(GSE_family)[sample_id]


In [26]:
def __sample_indices(GSE_family):
    
    """
    Exists to return the indices of each sample's information within the associated family dictionary. Returns a dictionary
    with keys equal to the sample ID ("GSM***") and values equal to the index of that sample's information within the family
    dictionary.
    """
    
    family_dir = __family_path(GSE_family)
    file_list = os.listdir(family_dir)
    
    filtered_list = []
    for file_name in file_list:
        sample_match = re.match(r"GSM", file_name)
        if sample_match:
            filtered_list.append(file_name)
            
    for i in np.arange(len(filtered_list)):
        filtered_list[i] = re.match(r"GSM\d+", filtered_list[i]).group(0)
        
    index_dict = {}
    index = 0

    for sample_id in filtered_list:
        index_dict[sample_id] = index
        index += 1
        
    return index_dict


In [27]:
def extract():
    
    """
    Will extract files from all downloaded family .tgz files to a respective directory: ./data/GSE***/
    
    This will also delete the .tgz and .gz files.
    """

    file_list = os.listdir(__base_path)
    tgz_list = []
    gz_list = []
    
    for filename in file_list:
        match_tgz = re.search(r"\.tgz", filename)
        match_gz = re.search(r"\.gz", filename)
        
        if match_tgz:
            tgz_list.append(filename)
        elif match_gz:
            gz_list.append(filename)
                
    flag = False
    
    for filename in tgz_list:
        full_path = __base_path + "/" + filename
        family_id = re.search(r"GSE\d+", filename).group(0)
                
        file = tarfile.open(full_path)
        
        try:
            out_path = "./" + __base_path + "/" + family_id
            file.extractall(out_path)
        except:
#             let's end the content extraction of the file. will fix later.
            file.close()
    
            flag = True
            print("Something weird happened while extracting from the " + family_id + " compressed file. Ended extraction early for " + family_id + ".")
            
        if not flag:
            file.close()
            os.remove(full_path)
            
    
    for filename in gz_list:
        full_path = "./" + __base_path + "/" + filename
        family_id = re.search(r"GSE\d+", filename).group(0)
                
        compressed_file = open(full_path, 'rb')
        compressed_file_contents = compressed_file.read()
        compressed_file.close()
        
        contents_bytes = gzip.decompress(compressed_file_contents)
        contents_str = contents_bytes.decode(errors= "replace")
        
        family_dir = os.path.join(__base_path, family_id)
        if not os.path.exists(family_dir):
            os.mkdir(family_dir)
        
        out_path = "./" + __base_path + "/" + family_id + "/" + family_id + "_series_matrix.txt"

        file = open(out_path, mode= 'w', encoding= "utf-8")
        
        file.write(contents_str)
        file.close()
    
        if not flag:
            file.close()
            os.remove(full_path)
    

In [30]:
series("GSE41826")


,GSM1024873,GSM1024874,GSM1024875,GSM1024876,GSM1024877,GSM1024878,GSM1024879,GSM1024880,GSM1024881,GSM1024882,...,GSM1025008,GSM1025009,GSM1025010,GSM1025011,GSM1025012,GSM1025013,GSM1025014,GSM1025015,GSM1025016,GSM1025018
ID_REF,,,,,,,,,,,,,,,,,,,,,
cg00000029,0.623261,0.511191,0.563197,0.540890,0.534855,0.594416,0.618680,0.709379,0.538162,0.647746,...,0.558410,0.573453,0.641118,0.559579,0.573836,0.588372,0.651390,0.599684,0.568034,0.590776
cg00000108,0.933737,0.947231,0.937874,0.936387,0.948110,0.933695,0.938969,0.946008,0.935681,0.948072,...,0.875346,0.925848,0.935957,0.868049,0.940325,0.933061,0.933560,0.915330,0.923610,0.915840
cg00000109,0.784065,0.783920,0.897161,0.827485,0.881682,0.784630,0.717540,0.854756,0.887938,0.822150,...,0.857118,0.807007,0.800926,0.859287,0.801134,0.826387,0.865034,0.836918,0.810071,0.880893
cg00000165,0.230605,0.144698,0.149871,0.142597,0.160029,0.242640,0.245693,0.268578,0.158757,0.278187,...,0.233702,0.234053,0.242062,0.293241,0.220576,0.209608,0.251258,0.213750,0.232467,0.228229
cg00000236,0.856642,0.821330,0.819246,0.748252,0.811866,0.882721,0.870907,0.896765,0.833451,0.891856,...,0.867157,0.848914,0.867830,0.831282,0.846274,0.847059,0.859088,0.854092,0.859797,0.867508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rs9363764,0.947018,0.927759,0.049844,0.611128,0.043295,0.055078,0.608171,0.936968,0.937653,0.950246,...,0.945493,0.950142,0.931523,0.944783,0.644050,0.946375,0.942450,0.928691,0.661740,0.067630
rs939290,0.958767,0.943658,0.949976,0.950231,0.945776,0.961935,0.960629,0.053784,0.065930,0.043154,...,0.953940,0.590611,0.944105,0.611173,0.647829,0.961473,0.039425,0.047130,0.953698,0.599880
rs951295,0.947758,0.952008,0.562691,0.538780,0.558405,0.530417,0.517637,0.505993,0.541859,0.498712,...,0.934698,0.508184,0.502462,0.537930,0.527069,0.953287,0.494407,0.516329,0.524182,0.936627
